In [1]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Linear
import torch

In [11]:
dataset=Planetoid(root="data/Planetoid",name='Cora',transform=NormalizeFeatures())

In [12]:
print(len(dataset))

1


In [13]:
data=dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [14]:
print(data.x[0][:50])
print(data.train_mask)
print(data.y)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
tensor([ True,  True,  True,  ..., False, False, False])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [15]:
print(data.edge_index.t())

tensor([[   0,  633],
        [   0, 1862],
        [   0, 2582],
        ...,
        [2707,  598],
        [2707, 1473],
        [2707, 2706]])


In [21]:
class GCN(torch.nn.Module):
    def __init__(self,hidden_channels):
        super(GCN,self).__init__()
        torch.manual_seed(42)
        
        self.conv1=GCNConv(dataset.num_features,hidden_channels)
        self.conv2=GCNConv(hidden_channels,hidden_channels)
        self.out=Linear(hidden_channels,dataset.num_classes)
        
    def forward(self,x,edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

     
        x = F.softmax(self.out(x), dim=1)
        return x

In [28]:
model=GCN(hidden_channels=16)
learning_rate=0.01
decay_rate=5e-4
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=decay_rate)
criteria=torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    
    out=model(data.x,data.edge_index)
    
    loss=criteria(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

In [30]:
def test():
    model.eval()
    out=model(data.x,data.edge_index)
    pred=out.argmax(dim=1)
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  
      
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  
    return test_acc,pred

    

In [44]:
losses=[]
for epoch in range(1,1000):
    loss=train()
    losses.append(loss)

In [45]:
losses

[tensor(1.3381, grad_fn=<NllLossBackward0>),
 tensor(1.3206, grad_fn=<NllLossBackward0>),
 tensor(1.3396, grad_fn=<NllLossBackward0>),
 tensor(1.3969, grad_fn=<NllLossBackward0>),
 tensor(1.3423, grad_fn=<NllLossBackward0>),
 tensor(1.3462, grad_fn=<NllLossBackward0>),
 tensor(1.3460, grad_fn=<NllLossBackward0>),
 tensor(1.3488, grad_fn=<NllLossBackward0>),
 tensor(1.3439, grad_fn=<NllLossBackward0>),
 tensor(1.3608, grad_fn=<NllLossBackward0>),
 tensor(1.3499, grad_fn=<NllLossBackward0>),
 tensor(1.3797, grad_fn=<NllLossBackward0>),
 tensor(1.3587, grad_fn=<NllLossBackward0>),
 tensor(1.3447, grad_fn=<NllLossBackward0>),
 tensor(1.3599, grad_fn=<NllLossBackward0>),
 tensor(1.3631, grad_fn=<NllLossBackward0>),
 tensor(1.3162, grad_fn=<NllLossBackward0>),
 tensor(1.3510, grad_fn=<NllLossBackward0>),
 tensor(1.3276, grad_fn=<NllLossBackward0>),
 tensor(1.3971, grad_fn=<NllLossBackward0>),
 tensor(1.3395, grad_fn=<NllLossBackward0>),
 tensor(1.3641, grad_fn=<NllLossBackward0>),
 tensor(1.

In [47]:
test_acc,pred=test()

In [53]:
print(test_acc)
print(pred)
print(data.y[data.test_mask])

0.766
tensor([3, 4, 4,  ..., 3, 3, 3])
tensor([3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
        2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 4, 4, 4, 4, 1, 1, 3, 1, 0, 3, 0,
        2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5,
        5, 5, 2, 2, 2, 2, 1, 6, 6, 3, 0, 0, 5, 0, 5, 0, 3, 5, 3, 0, 0, 6, 0, 6,
        3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 4, 4, 4, 0, 3, 3, 2, 5, 5, 5, 5,
        6, 5, 5, 5, 5, 0, 4, 4, 4, 0, 0, 5, 0, 0, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0,
        3, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4,
        6, 6, 5, 6, 6, 3, 5, 5, 5, 0, 5, 0, 4, 4, 3, 3, 3, 2, 2, 1, 3, 3, 3, 3,
 